# Private Prediction Client

In [1]:
import numpy as np
import tensorflow as tf
import tf_encrypted as tfe

from tensorflow.keras.datasets import mnist

/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Set up `tfe.serving.QueueClient`

In [3]:
config = tfe.RemoteConfig.load("/tmp/tfe.config")

tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

In [4]:
input_shape = (1, 784)
output_shape = (1, 10)

In [5]:
client = tfe.serving.QueueClient(
    input_shape=input_shape,
    output_shape=output_shape)

In [6]:
sess = tfe.Session(config=config)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## Query Model

In [7]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [8]:
for image, expected_label in zip(images, expected_labels):
    
    res = client.run(
        sess,
        image.reshape(1, 784))
    
    predicted_label = np.argmax(res)
    
    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "incorrectly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1
